## LSTM과 GridSearchCV를 이용한 예측 코드 (Python)
```
중요 경고: 번호는 독립적이고 무작위적인 추첨으로 결정됩니다. 과거의 데이터로 미래의 번호를 예측하는 것은 통계적으로 불가능하며, 이 코드는 오락 및 학습 목적으로만 사용해야 합니다.
```

<p>다음은 LSTM(Long Short-Term Memory) 신경망과 GridSearchCV를 사용하여 CSV 파일에 저장된 과거 번호를 읽어 다음 번호를 "예측" (시퀀스 패턴 학습 시도)하는 예제 코드입니다.</p>

### 사전 준비:

1. 데이터 파일 (history.csv):

 * CSV 파일에는 과거 번호가 들어 있어야 합니다.
 * 예상되는 형식: 각 행은 하나의 추첨 결과를 나타내며, 번호는 오름차순으로 정렬되어 있고, 'num1', 'num2', 'num3', 'num4', 'num5', 'num6'과 같은 열 이름을 가진다고 가정합니다. (보너스 번호는 이 예제에서 제외)
 * 예시 (history.csv):
```
회차,날짜,num1,num2,num3,num4,num5,num6
1118,2024-05-04,2,10,15,22,30,41
1117,2024-04-27,5,12,18,25,33,42
... (더 많은 데이터)
2. 필수 라이브러리 설치:
```Bash
pip install pandas numpy tensorflow scikit-learn scikeras
```
 * pandas: CSV 파일 처리
 * numpy: 수치 연산
 * tensorflow: Keras API를 통해 LSTM 모델 구축
 * scikit-learn: 데이터 전처리 (MinMaxScaler), 모델 평가 (GridSearchCV)
 * scikeras: Keras 모델을 Scikit-learn과 호환되도록 래핑

### Python 코드:

In [4]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
!pip install --upgrade scikeras tensorflow-cpu "xgboost>=2.1.4" "scikit-learn==1.5.2"
restart_kernel()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.9/251.9 MB 15.1 MB/s eta 0:00:0000:0100:01


In [57]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Or "-1"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Verify that GPU is not being used
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
# # Verify that GPU is not being used
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
import warnings

# 경고 메시지 무시 (KerasRegressor 관련 경고가 나올 수 있음)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [59]:
# --- 설정값 ---
FILE_PATH = './selectd1.csv'  # CSV 파일 경로
# CSV 파일에서 실제 번호가 시작되는 열 이름들 (num1 ~ num6)
NUMBER_COLUMNS = [f'{i}' for i in range(1, 7)]
SEQUENCE_LENGTH = 10  # 이전 몇 회차의 데이터를 보고 다음 회차를 예측할지 결정
N_FEATURES = len(NUMBER_COLUMNS) # 예측할 번호의 개수 (6개)
NUM_MIN = 1
NUM_MAX = 45

In [60]:
# --- 1. 데이터 로드 및 전처리 ---
def load_and_preprocess_data(file_path, number_columns, sequence_length, scaler):
    try:
        df = pd.read_csv(file_path)
        df = df.sort_values(by='round')
    except FileNotFoundError:
        print(f"오류: 파일 '{file_path}'을(를) 찾을 수 없습니다.")
        print("history.csv 파일을 생성하고 번호를 넣어주세요.")
        print("예시 형식:\n회차,날짜,num1,num2,num3,num4,num5,num6\n1118,2024-05-04,2,10,15,22,30,41")
        return None, None, None, None, None

    # 번호 열만 선택하고, 오름차순으로 정렬되어 있다고 가정
    # 만약 정렬되어 있지 않다면, 각 행을 정렬하는 코드를 추가해야 합니다.
    # 예: df[number_columns] = np.sort(df[number_columns].values, axis=1)
    numbers = df[number_columns].values

    if len(numbers) < sequence_length + 1:
        print(f"오류: 데이터가 너무 적습니다. 최소 {sequence_length + 1}개의 추첨 기록이 필요합니다.")
        return None, None, None, None, None

    # 데이터 스케일링 (0과 1 사이로)
    # 각 번호는 1~45 범위이므로, 이를 기준으로 스케일링
    
    # 스케일러 학습을 위해 1차원 배열로 변환 후 다시 원래 형태로
    scaled_numbers = scaler.fit_transform(numbers.reshape(-1, 1)).reshape(numbers.shape)

    X, y = [], []
    for i in range(len(scaled_numbers) - sequence_length):
        X.append(scaled_numbers[i:(i + sequence_length)]) # 이전 sequence_length 만큼의 데이터
        y.append(scaled_numbers[i + sequence_length])    # 다음 회차 데이터

    X = np.array(X)
    y = np.array(y)

    if X.shape[0] == 0:
        print("오류: 시퀀스 데이터를 생성할 수 없습니다. 데이터 양이나 sequence_length를 확인하세요.")
        return None, None, None, None, None

    return X, y, scaler, numbers # numbers는 마지막 시퀀스 생성에 사용

In [61]:
# --- 2. LSTM 모델 생성 함수 ---
def create_lstm_model_callback(lstm_units=50,
                               dense_units=25,
                               learning_rate=0.001,
                               sequence_length=SEQUENCE_LENGTH,
                               n_features=N_FEATURES):
    model = Sequential([
        Input(shape=(sequence_length, n_features)),
        LSTM(lstm_units, activation='relu', return_sequences=True), # 여러 LSTM 층을 쌓을 경우 True
        LSTM(int(lstm_units/2), activation='relu'),
        Dense(dense_units, activation='relu'),
        Dense(n_features) # 출력층: 6개 번호, 활성화 함수 없음 (회귀)
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse') # 평균 제곱 오차 손실 함수
    return model

In [138]:
def create_lstm_model(X_data, y_data):
    # 데이터 분할 (훈련셋, 테스트셋)
    # GridSearchCV가 교차검증을 하므로, 별도의 검증셋은 생략 가능.
    # 테스트셋은 최종 모델 평가에 사용.
    if len(X_data) < 20: # GridSearchCV를 위한 최소 데이터양 (임의로 설정)
        print("데이터가 GridSearchCV를 수행하기에 너무 적습니다. 더 많은 데이터를 확보해주세요.")
        X_train, y_train = X_data, y_data
        X_test, y_test = X_data[-1:], y_data[-1:] # 임시로 마지막 데이터 사용
        print("GridSearchCV 없이 기본 모델로 학습 및 예측을 시도합니다.")
        perform_grid_search = False
    else:
        X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                            y_data,
                                                            test_size=0.2,
                                                            random_state=42,
                                                            shuffle=False)
        perform_grid_search = True

    if perform_grid_search:
        print(f"\n훈련 데이터 형태: {X_train.shape}, {y_train.shape}")
        print(f"테스트 데이터 형태: {X_test.shape}, {y_test.shape}")

        # Keras 모델을 scikit-learn과 호환되도록 래핑
        # KerasRegressor에 전달되는 파라미터는 create_lstm_model 함수의 파라미터 이름과 일치해야 함
        # 혹은 model__ 접두사를 사용하여 모델 내부 파라미터 지정 가능
        model_kr = KerasRegressor(
            model=create_lstm_model_callback,
            # create_lstm_model 함수에 전달될 고정 파라미터
            lstm_units=50,
            dense_units=25,
            learning_rate=0.001, 
            sequence_length=SEQUENCE_LENGTH,
            n_features=N_FEATURES,
            verbose=0 # 학습 중 로그 출력 안 함
        )
        print(f'perform_grid_search = {model_kr.get_params()}')
        # GridSearchCV를 위한 하이퍼파라미터 그리드 정의
        # 주의: 너무 많은 조합은 매우 긴 학습 시간을 유발할 수 있습니다.
        #'lstm_units': [32, 64],          # LSTM 유닛 수
        # 'dense_units': [16, 32],         # Dense 유닛 수
        #'batch_size': [1, 16, 32],          # 배치 크기
        # param_grid = {
        #     'lstm_units': [1],
        #     'dense_units': [1],
        #     'learning_rate': [0.001], # 학습률
        #     'batch_size': [1],
        #     'epochs': [1]# 에포크 수
        # }
        param_grid = {
            'lstm_units': [90, 45, 20, 10],
            'dense_units': [90, 45, 20, 10],
            'learning_rate': [0.001, 0.002, 0.005], # 학습률
            'batch_size': [1, 3, 6],
            'epochs': [20, 50, 100, 200]              # 에포크 수
        }
        # 실제 사용시에는 더 적은 조합으로 시작하거나, RandomizedSearchCV 사용 고려

        print("\nGridSearchCV를 시작합니다. 시간이 오래 걸릴 수 있습니다...")
        grid_search = GridSearchCV(estimator=model_kr,
                                   param_grid=param_grid,
                                   # cv=2, # 교차 검증 폴드 수 (데이터가 적으면 줄이세요)
                                   cv=3,
                                   scoring='neg_mean_squared_error', # 점수 기준 (낮을수록 좋음)
                                   verbose=1, # 진행 상황 출력
                                   n_jobs=-1) # 사용 가능한 모든 CPU 코어 사용

        grid_search.fit(X_train, y_train)

        print("\nGridSearchCV 완료.")
        print(f"최적 하이퍼파라미터: {grid_search.best_params_}")
        print(f"최적 점수 (Negative MSE): {grid_search.best_score_:.4f}")

        best_model = grid_search.best_estimator_
    else: # GridSearchCV를 수행하지 않는 경우
        print("\n기본 모델로 학습을 시작합니다...")
        # 기본 파라미터로 모델 생성 및 학습
        best_model = create_lstm_model_local() # 기본값 사용
        best_model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
    return best_model, X_test, y_test

In [139]:
def gen_random_with_predicted(predicted_value, trial=1, verb="verbose", use_pre=True):
    prediction_number_set = []
    # yhat = cls_object.model.predict(cls_object.test_X) # [1x45] dim
    # probability = cls_object.get_probability(list(yhat[0])) # use the output as prob. desity dist.
    probability = predicted_value[0]
    print(f'pre -1 - predicted_value = {predicted_value}, {type(predicted_value)}')
    print(f'pre 0 - probability={probability}')
    for t in range(1, trial+1):
        selected = np.random.choice(NUM_MAX, size=6, replace=False, p=probability)
        prediction_number_set.append([int(i) + 1 for i in selected])
        if verb == 'verbose':
            print("predicted set of numbers at {}th trial :".format(t), selected + 1)
        if use_pre:
            probabality = selected
        print(f'after {t} - probability={probability}')            
    return prediction_number_set

In [140]:
def get_precicted_scale(X_data, best_model):
    # 가장 최근 SEQUENCE_LENGTH 개의 데이터를 사용하여 다음 번호 예측
    last_sequence_scaled = X_data[-1].reshape(1,
                                              SEQUENCE_LENGTH,
                                              N_FEATURES)

    # 모델을 사용하여 예측 (결과는 0~1 사이로 스케일링된 값)
    predicted_scaled = best_model.predict(last_sequence_scaled)
    return predicted_scaled

In [141]:
def get_predicted_value(predicted_scaled, scaler):
    # 예측값을 원래 스케일로 되돌림
    # scaler는 (n_samples, n_features) 형태를 기대하므로, 예측값을 적절히 변형
    # 각 번호가 독립적으로 스케일링 되었으므로, 예측된 6개 값에 대해 각각 inverse_transform 적용
    # 여기서는 전체 번호 데이터(1~45)로 학습된 scaler를 사용하므로,
    # 예측된 6개 값을 1차원으로 풀고, 다시 2차원으로 만들어 inverse_transform 후, 다시 1차원으로 풀고, 정수 변환
    predicted_numbers_flat = scaler.inverse_transform(predicted_scaled.reshape(-1, 1)).flatten()

    # 정수로 변환하고 1~45 범위로 클리핑
    predicted_numbers = np.round(predicted_numbers_flat).astype(int)
    predicted_numbers = np.clip(predicted_numbers, NUM_MIN, NUM_MAX)

    # 예측된 번호들을 정렬하고, 중복 제거 (선택적)
    # LSTM 모델은 중복된 번호를 예측할 수 있습니다. 실제 로또는 중복이 없으므로 처리 필요.
    # 가장 간단한 방법: set으로 변환 후 다시 list로. 만약 6개 미만이 되면 추가 로직 필요
    unique_predicted_numbers = sorted(list(set(predicted_numbers)))

    # 만약 unique_predicted_numbers 개수가 6개 미만이면,
    # 부족한 만큼 다른 번호로 채우는 로직이 필요할 수 있습니다.
    # (예: 예측 확률이 가장 높았지만 중복으로 제외된 다음 번호, 또는 랜덤 번호 등)
    # 이 예제에서는 단순 출력합니다.
    if len(unique_predicted_numbers) < N_FEATURES:
        print(f"\n경고: 예측된 고유 번호가 {N_FEATURES}개 미만입니다: {unique_predicted_numbers}")
        # 부족한 개수만큼 임의의 번호(1~45 중 예측된 번호 제외)로 채우기 (단순 예시)
        needed = N_FEATURES - len(unique_predicted_numbers)
        possible_fillers = [i for i in range(NUM_MIN, NUM_MAX + 1) if i not in unique_predicted_numbers]
        np.random.shuffle(possible_fillers) # 무작위로 섞음
        unique_predicted_numbers.extend(possible_fillers[:needed])
        unique_predicted_numbers = sorted(unique_predicted_numbers[:N_FEATURES])
    return unique_predicted_numbers

In [142]:
def predict(X_data, best_model, scaler, trial=1):
    predicted_scaled = get_precicted_scale(X_data, best_model)
    unique_predicted_numbers = []
    for i in range(trial):
        predicted_value = get_predicted_value(predicted_scaled, scaler)
        unique_predicted_numbers.append([i.item() for i in predicted_value])
    rnd_predicted = []
    # rnd_predicted = gen_random_with_predicted(predicted_scaled, trial=trial)
    return unique_predicted_numbers, rnd_predicted

In [143]:
def main(file_path, number_columns, trial=1):
    print("(LSTM + GridSearchCV) - 학습 및 교육용입니다.")
    scaler = MinMaxScaler(feature_range=(0, 1))

    X_data, y_data, scaler, original_numbers = load_and_preprocess_data(file_path,
                                                                        number_columns,
                                                                        SEQUENCE_LENGTH,
                                                                        scaler=scaler
                                                                       )
    if X_data is None:
        exit()

    expected = []

    best_model, X_test, y_test = create_lstm_model(X_data=X_data, y_data=y_data)
    # --- 4. 다음 번호 예측 ---
    expected, rnd_predicted = predict(X_data=X_data,
                                      best_model=best_model,
                                      scaler=scaler,
                                      trial=trial
                                      )

    print("\n--- 최종 예측 결과 ---")
    print(f"가장 최근 {SEQUENCE_LENGTH}개의 실제 번호들 (참고용):")
    # 마지막 SEQUENCE_LENGTH 개의 원본 번호 출력
    # original_numbers는 모든 원본 데이터를 포함
    last_n_draws = original_numbers[-(SEQUENCE_LENGTH + 1) : -1] # 예측에 사용된 입력 시퀀스
    for i, draw in enumerate(last_n_draws):
        print(f"  T-{SEQUENCE_LENGTH-i}: {draw}")
    print(f"  실제 마지막 번호 (T): {original_numbers[-1]}")
    print(f"\n다음 예측 번호 (LSTM + GridSearchCV):")
    for exp in expected:
        print(f"\t\t{exp}")
    print("______")
    for exp in rnd_predicted:
        print(f"\t\t{exp}")

    # (선택) 테스트 데이터로 모델 성능 평가 (MSE)
    if X_test.shape[0] > 0 and y_test.shape[0] > 0:
        test_loss = best_model.evaluate(X_test, y_test, verbose=0) if hasattr(best_model, 'evaluate') else "N/A (KerasRegressor)"
        if isinstance(test_loss, str): # KerasRegressor의 경우 evaluate 없음
             y_pred_test_scaled = best_model.predict(X_test)
             mse_test = np.mean((y_test - y_pred_test_scaled)**2)
             print(f"\n테스트 데이터에 대한 최종 모델의 MSE: {mse_test:.6f}")
        else:
             print(f"\n테스트 데이터에 대한 최종 모델의 MSE: {test_loss:.6f}")

In [144]:
print(f'Column Nums = {NUMBER_COLUMNS}')
main(file_path=FILE_PATH, number_columns=NUMBER_COLUMNS, trial=5)

Column Nums = ['1', '2', '3', '4', '5', '6']
(LSTM + GridSearchCV) - 학습 및 교육용입니다.

훈련 데이터 형태: (448, 10, 6), (448, 6)
테스트 데이터 형태: (112, 10, 6), (112, 6)
perform_grid_search = {'model': <function create_lstm_model_callback at 0x74e81f9b5e40>, 'build_fn': None, 'warm_start': False, 'random_state': None, 'optimizer': 'rmsprop', 'loss': None, 'metrics': None, 'batch_size': None, 'validation_batch_size': None, 'verbose': 0, 'callbacks': None, 'validation_split': 0.0, 'shuffle': True, 'run_eagerly': False, 'epochs': 1, 'lstm_units': 50, 'dense_units': 25, 'learning_rate': 0.001, 'sequence_length': 10, 'n_features': 6}

GridSearchCV를 시작합니다. 시간이 오래 걸릴 수 있습니다...
Fitting 3 folds for each of 576 candidates, totalling 1728 fits


2025-05-12 04:38:09.500543: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-12 04:38:22.136434: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-12 04:56:19.774306: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-12 04:56:22.127505: I tensorflow/core/platform/cpu_featu


GridSearchCV 완료.
최적 하이퍼파라미터: {'batch_size': 6, 'dense_units': 20, 'epochs': 50, 'learning_rate': 0.001, 'lstm_units': 10}
최적 점수 (Negative MSE): -0.0230

--- 최종 예측 결과 ---
가장 최근 10개의 실제 번호들 (참고용):
  T-10: [ 7 13 18 36 39 45]
  T-9: [ 2 12 20 24 34 42]
  T-8: [20 21 22 25 28 29]
  T-7: [ 2 13 15 16 33 43]
  T-6: [17 18 23 25 38 39]
  T-5: [ 6  7 27 29 38 45]
  T-4: [14 23 25 27 29 42]
  T-3: [ 8 23 31 35 39 40]
  T-2: [ 9 21 24 30 33 37]
  T-1: [ 5 12 24 26 39 42]
  실제 마지막 번호 (T): [ 3 13 28 34 38 42]

다음 예측 번호 (LSTM + GridSearchCV):
		[6, 13, 19, 24, 31, 38]
		[6, 13, 19, 24, 31, 38]
		[6, 13, 19, 24, 31, 38]
		[6, 13, 19, 24, 31, 38]
		[6, 13, 19, 24, 31, 38]
______

테스트 데이터에 대한 최종 모델의 MSE: 0.022148


In [94]:
# 1171회 당첨번호는 3 6 7 11 12 17